<a href="https://colab.research.google.com/github/z230501a/Padoru/blob/master/Resurrecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

利用機器學習模仿魔戒(朱學恆譯版)的文筆，並且給予開頭引導其能夠依照原文的思維接續下去。

程式參考自[穿越時空的偉人：用PyTorch重現偉人們的神經網絡](https://medium.com/pyladies-taiwan/%E7%A9%BF%E8%B6%8A%E6%99%82%E7%A9%BA%E7%9A%84%E5%81%89%E4%BA%BA-%E7%94%A8pytorch%E9%87%8D%E7%8F%BE%E5%81%89%E4%BA%BA%E5%80%91%E7%9A%84%E7%A5%9E%E7%B6%93%E7%B6%B2%E7%B5%A1-bd045ac43e96?fbclid=IwAR2I3JlAGpjCxhDckpyI8nCFdeVr2uBm_orDVfOsQt2xncXlXPrCjUFuCwY)這篇文章，並且進行修改後將其使用於Colabotary上。



---



而為了進行，所以要先導入套件。所用的套件有以下，這次主要用到的是Pytorch。

In [0]:
import pickle

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from io import BytesIO

**Pytorch介紹**

Pytorch是一種機器學習框架，由Facebook開源，建立在Torch之上，然而已經與Torch相差甚大。

標榜Python First，使用上與Python無異，並且能夠與其他Python套件整合。

學習的模式是，依照前面出現的句子，然後去找適合接續的字接續。

In [0]:



def parse_corpus(path, seq_length=50):
    '''Parse raw corpus text into input-output pairs, where input is a sequence of characters, output is 1 character after the sequence'''

    # Read text
    with open(path, 'r') as f:
        raw_text = f.read().replace('\n', '')

    # Get unique characters
    chars = sorted(list(set(raw_text)))

    # Map char to int / int to char
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    int_to_char = dict((i, c) for i, c in enumerate(chars))
    
    # Prepare training data, for every <seq_length> chars, predict 1 char after the sequence
    n_chars = len(raw_text)
    dataX = [] # N x seq_length
    dataY = [] # N x 1
    for i in range(0, n_chars - seq_length):
        seq_in = raw_text[i:i + seq_length]
        seq_out = raw_text[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
    
    return (dataX, dataY, char_to_int, int_to_char, chars)

def format_data(dataX, dataY, n_classes, batch_size=64):
    '''Parse into minibatches, return Tensors'''

    # For simplicity, discard trailing data not fitting into batch_size
    n_patterns = len(dataY)
    n_patterns = n_patterns - n_patterns % batch_size
    X = dataX[:n_patterns]
    Y = dataY[:n_patterns]

    # Parse X
    X = np.array(X)
    _, seq_length = X.shape
    X = X.reshape(-1, batch_size, seq_length)

    X = torch.LongTensor(X)

    # Parse Y
    Y = np.array(Y)
    Y = Y.reshape(-1, batch_size)

    Y = torch.LongTensor(Y)

    return list(zip(X, Y))

In [0]:



class Net(nn.Module):
    def __init__(self, n_vocab, embedding_dim, hidden_dim, dropout=0.2):
        super(Net, self).__init__()

        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        self.embeddings = nn.Embedding(n_vocab, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, dropout=dropout)
        self.hidden2out = nn.Linear(hidden_dim, n_vocab)

    def forward(self, seq_in):
        embeddings = self.embeddings(seq_in.t()) # LSTM takes 3D inputs (timesteps, batch, features)
                                                 #                    = (seq_length, batch_size, embedding_dim)
        lstm_out, _ = self.lstm(embeddings)      # Each timestep outputs 1 hidden_state
                                                 # Combined in lstm_out = (seq_length, batch_size, hidden_dim) 
        ht = lstm_out[-1]                        # ht = last hidden state = (batch_size, hidden_dim)
                                                 # Use the last hidden state to predict the following character
        out = self.hidden2out(ht)                # Fully-connected layer, predict (batch_size, n_vocab)

        return out

In [0]:



def load_pickle(path):
    with open(path,'rb') as f:
      data = pickle.load(f)
      f.close()
    return data

def is_end(c):
    end_tokens = ['。', '？', '！', '.', '?', '!']
    return c in end_tokens

def to_prob(vec):
    s = sum(vec)
    return [v / s for v in vec]

def gen_text(model, patterns, char_to_int, int_to_char, chars, n_sent=10,restart_seq=False):
    n_patterns = len(patterns)

    # Randomly choose a pattern to start text generation
    start=np.random.randint(0, n_patterns - 1)
    #start = np.random.randint(0, n_patterns - 1)
    pattern = patterns[start]
    # Start generation until n_sent sentences generated 
    cnt = 0
    while cnt < n_sent: 
        # Format input pattern
        seq_in = np.array(pattern)
        seq_in = seq_in.reshape(1, -1) # batch_size = 1

        seq_in = Variable(torch.LongTensor(seq_in))

        # Predict next character
        pred = model(seq_in)
        pred = to_prob(F.softmax(pred, dim=1).data[0].numpy()) # turn into probability distribution
        char = np.random.choice(chars, p=pred)                 # pick char based on probability instead of always picking the highest value
        char_idx = char_to_int[char]
        print(char, end='')

        # Append predicted character to pattern, truncate to usual pattern size, use as new pattern
        pattern.append(char_idx)
        pattern = pattern[1:]

        if is_end(char):
            if restart_seq:
                start = np.random.randint(0, n_patterns - 1)
                pattern = patterns[start]
                print()

            cnt += 1 
    
    if not restart_seq:
        print()

In [0]:


def load_data(path, seq_length, batch_size):
    dataX, dataY, char_to_int, int_to_char, chars = parse_corpus(path, seq_length=seq_length)
    data = format_data(dataX, dataY, n_classes=len(chars), batch_size=batch_size)

    return data, dataX, dataY, char_to_int, int_to_char, chars

def save_pickle(data, path):
    with open(path, 'wb') as f:  
        pickle.dump(data, f)
        f.close()

def train(model, optimizer, epoch, data, log_interval):
    model.train()

    for batch_i, (seq_in, target) in enumerate(data):
        seq_in, target = Variable(seq_in), Variable(target)
        optimizer.zero_grad()

        output = model(seq_in)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        # Log training status
        if batch_i % log_interval == 0:
            print('Train epoch: {} ({:2.0f}%)\tLoss: {:.6f}'.format(epoch, 100. * batch_i / len(data), loss.data))

In [0]:
#@title


if __name__ == '__main__':

    # Prepare
    train_data, dataX, dataY, char_to_int, int_to_char, chars = load_data("corpus.txt", seq_length=8, batch_size=16)
    model = Net(len(chars), 2, 254, dropout=0.3)
    #Net(len(chars),args.embedding_dim, args.hidden_dim, dropout=args.dropout)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    #torch.optim.Adam(model.parameters(), lr=args.lr)

    # Train
    for epoch in range(100):
    #for epoch in range(args.epochs)
        train(model, optimizer, epoch, train_data, log_interval=10)
        #train(model, optimizer, epoch, train_data, log_interval=args.log_interval)

        if (epoch + 1) % 10 == 0:
          #(epoch + 1) % args.save_interval == 0
            model.eval()
            torch.save(model.state_dict(), "training_backup.pkl")#僅儲存參數，而非整個模型
            #torch.save(model, args.output)

 # Save mappings, vocabs, & model
    save_pickle((dataX, char_to_int, int_to_char, chars), "training_save.pkl")
    #save_pickle((dataX, char_to_int, int_to_char, chars), args.output_c)

    model.eval()
    torch.save(model, "training_model.pkl")#儲存整個模型
    
    # Load model
    model=torch.load("training_model.pkl")

    # Load mappings & vocabularies
    dataX, char_to_int, int_to_char, chars = load_pickle("training_save.pkl")
    
    # Generate text
    gen_text(model, dataX, char_to_int, int_to_char, chars, n_sent=10, restart_seq=False)

In [0]:
#讀取備分

    # Prepare
    train_data, dataX, dataY, char_to_int, int_to_char, chars = load_data("corpus.txt", seq_length=8, batch_size=32)
    model.load_state_dict(torch.load("training_backup.pkl"))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    # Train
    for epoch in range(10):

        train(model, optimizer, epoch, train_data, log_interval=10)


        if (epoch + 1) % 10 == 0:

            model.eval()
            torch.save(model, "training_backup.pkl")


 # Save mappings, vocabs, & model
    save_pickle((dataX, char_to_int, int_to_char, chars), "training_save.pkl")
    #save_pickle((dataX, char_to_int, int_to_char, chars), args.output_c)

    model.eval()
    torch.save(model, "training_model.pkl")
    

In [0]:
#產生文本
if __name__ == '__main__':
 # Load model
    model=torch.load("training_model.pkl")

    # Load mappings & vocabularies
    dataX, char_to_int, int_to_char, chars = load_pickle("training_save.pkl")
    
    # Generate text
    gen_text(model, dataX, char_to_int, int_to_char, chars, n_sent=10, restart_seq=False)

In [0]:
#取得模型資料
print(load_pickle("training_save.pkl"))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

